In [76]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import statsmodels.formula.api as sm
import os
import numpy as np
from functools import reduce
import matplotlib as mpl
%matplotlib inline

In [ ]:
pd.set_option('max_colwidth',250)
pd.set_option('display.width',250)

In [12]:
os.chdir('/home/matt/Git/PGIP/src')
os.getcwd()

'/home/matt/Git/PGIP/src'

In [ ]:
import pipeline as PGIP
os.chdir('..')
E = PGIP.Explorer('config/cache')
P = PGIP.Pipeline('config/cache')

In [ ]:
E.readSampleData('sandbox')
E.renameColumns()

In [27]:
df = pd.DataFrame()
for source in E.data:
    data = E.data[source].data
    data['Source'] = source
    df = pd.concat([df,data])

In [28]:
df.shape

(2385, 33)

In [ ]:
df['Month'] = df.apply(P.adders.addMonth,axis=1)

In [80]:
df.Month.isnull().sum()

995

In [79]:
df.Year.isnull().sum()

0

In [ ]:
df['TimeSlice'] = df.apply(P.adders.addTimeSlice,axis = 1)

In [ ]:
df.TimeSlice.isnull().sum()

In [ ]:
df.Issue = df.Issue.apply(P.cleaners.cleanIssue)

In [57]:
df.Volume = df.Volume.apply(P.cleaners.cleanVolume)

In [92]:
def pred(row,params,target):
    
    if pd.isnull(row[target]):
        row.loc['Intercept'] = 1
        row = row[params.index]
        if any([pd.isnull(i) for i in row]):
            return np.nan
        
        p = row*params
        p = round(p.sum(),0)
        
        return int(p)
        
    else:
        return row.loc[target]

In [105]:
codens = df.CODEN.unique()
confidenceThreshold = .6


for code in codens:    
    a = df.loc[df.CODEN == code,['AbbrPubTitle','PublicationTitle','Issue','Volume','Year','Month','TimeSlice']]
    abbrevs = a.AbbrPubTitle.unique()
    titles = a.PublicationTitle.unique()
    
    if len(abbrevs) > 1:
        df.loc[df.AbbrPubTitle.isin(abbrevs) | df.PublicationTitle.isin(titles),'CODEN'] = code
        a = df.loc[df.CODEN == code,['AbbrPubTitle','Issue','Volume','Year','Month','TimeSlice','Source']]
    if a.shape[0] < 2:
        continue
        
    count = a.TimeSlice.isnull().sum()
    #print(count)
    if count == 0:
        continue
    
    if (a.Issue.notnull() & a.Volume.notnull() & a.TimeSlice.notnull()).sum() >= 3:
        imputable = (a.Issue.notnull() & a.Volume.notnull() & a.TimeSlice.isnull()).sum()
        if imputable > 0:
            result = sm.ols(formula="TimeSlice ~ Volume + Issue", data=a).fit()
            target = 'TimeSlice'
    elif (a.Issue.notnull() & a.Month.notnull()).sum() >= 2:
        imputable = (a.Issue.notnull() & a.Month.isnull()).sum()
        if imputable > 0:
            result = sm.ols(formula="Month ~ Issue", data=a).fit()
            target = 'Month'
    elif (a.Volume.notnull() & a.TimeSlice.notnull()).sum() >= 2:
        imputable = (a.Volume.notnull() & a.TimeSlice.isnull()).sum()
        if imputable > 0:
            result = sm.ols(formula="TimeSlice ~ Volume", data=a).fit()
            target = 'TimeSlice'
    else:
        continue
    
    if result.rsquared > confidenceThreshold:
        p = a.apply(pred,axis =1,**{'params':result.params,'target':target})
        a.loc[:,target] = p
        print(a)
        if target == 'Month':
            a.loc[:,'TimeSlice'] = a.apply(lambda row: 12*(row.loc['Year'] - 1990) + row.loc['Month'])
        

         AbbrPubTitle      PublicationTitle  Issue  Volume  Year  Month  TimeSlice
2    Mar. Biotechnol.  Marine Biotechnology     30      17  2015      6        306
41   Mar. Biotechnol.  Marine Biotechnology    NaN     NaN  2015      5        305
44   Mar. Biotechnol.  Marine Biotechnology     20      17  2015    NaN        304
106  Mar. Biotechnol.  Marine Biotechnology     10      16  2014    NaN        290
137  Mar. Biotechnol.  Marine Biotechnology     20      15  2013    NaN        280
159  Mar. Biotechnol.  Marine Biotechnology     60      14  2012    NaN        276
168  Mar. Biotechnol.  Marine Biotechnology     10      14  2012      2        266
184  Mar. Biotechnol.  Marine Biotechnology     60      15  2013     12        288
187  Mar. Biotechnol.  Marine Biotechnology     20      15  2013    NaN        280
189  Mar. Biotechnol.  Marine Biotechnology     30      15  2013    NaN        282
235  Mar. Biotechnol.  Marine Biotechnology     30      11  2009      6        234
239 

In [103]:
a.

Int64Index([2, 41, 44, 106, 137, 159, 168, 184, 187, 189, 235, 239, 600, 688], dtype='int64')

In [69]:
result.params.index

Index(['Intercept', 'Volume', 'Issue'], dtype='object')

In [39]:

    #print(a)
    if (a.Issue.notnull() & a.Volume.notnull() & a.month.notnull()).sum() >= 3:
        if (a.Issue.notnull() & a.Volume.notnull() & a.month.isnull()).sum() > 0:
            result = LinearRegression()
            #print(a)
            #result.fit(a.loc[a.Issue.notnull() & a.Volume.notnull() & a.month.isnull(),['Issue','Volume']],a.loc[a.Issue.notnull() & a.Volume.notnull() & a.month.isnull(),'month'])
            print('sad')
            #a.loc[:,'month'] = result.predict()
            #print(result.predict(a.loc[(a.Issue.notnull() & a.Volume.notnull() & a.Month.isnull()),:]))
    elif (a.Issue.notnull() & a.Month.notnull()).sum() >= 2:
        if (a.Issue.notnull() & a.Month.isnull()).sum() > 0:
            #result = sm.ols(formula="Month ~ Issue", data=a).fit()
            print('soup')
            print(result.rsquared)
            a.loc[:,['Month']] = a.apply(pred,(result.params,'Month'))
    elif (a.Volume.notnull() & a.month.notnull()).sum() >= 2:
        if (a.Volume.notnull() & a.month.isnull()).sum() > 0:
            #result = sm.ols(formula="month ~ Volume", data=a).fit()
            print('laf')
            print(result.rsquared)
            #a.loc[:,'month'] = result.predict()
    else:
        continue

IndentationError: unexpected indent (<ipython-input-39-b35025a86a67>, line 3)

True

In [109]:
def imputeCODEN(connection,table,recordID=None):
    if not recordID:
        recordID = 'ROWID'
    columns = [recordID,'CODEN','AbbrPubTitle','PublicationTitle']
    columnString = ', '.join(columns)

    codens = connection.execute('SELECT DISTINCT CODEN FROM {0}'.format(table)).fetchall()
    codens = list(next(zip(*codens)))
    
    for code in codens:
        
        command = 'SELECT {0} FROM {1} WHERE CODEN = "{3}"'.format(columnString,table,code)
        df = connection.execute(command).fetchall()
        df = pd.DataFrame(df,columns = columns)
        
        abbrevs = list(df.AbbrPubTitle.unique())
        titles = list(df.PublicationTitle.unique())

        if len(abbrevs) > 1 or len(titles) > 1:
            # Update the db
            abbrString = '"' + '","'.join(abbrevs) + '"'
            titleString = '"' + '", "'.join(titles) + '"'
            
            command = ('UPDATE {0} SET CODEN = "{1}" ' +
                      'WHERE AbbrPubTitle IN ({2}) ' +
                      'OR PublicationTitle IN ({3})').format(table,code,abbrString,titleString)
            
            mem.execute(command)
        

def imputeTimeSlice(connection,table,recordID=None,confidenceThreshold = .6):
    if not recordID:
        recordID = 'ROWID'
    columns = [recordID,'Issue','Volume','Year','Month','TimeSlice']
    columnString = ', '.join(columns)
        
    codens = connection.execute('SELECT DISTINCT CODEN FROM {0}'.format(table)).fetchall()
    codens = list(next(zip(*codens)))
    
    for code in codens:
        command = 'SELECT {0} FROM {1} WHERE CODEN = "{3}"'.format(columnString,table,code)
        df = connection.execute(command).fetchall()
        df = pd.DataFrame(df,columns = columns) 

        if df.Timeslice.isnull().sum() == 0 or df.shape[0] < 2:
            continue     
        
        df = imputeTimeSliceSample(df)
        df = df[df.imputed,[recordID,'Month','TimeSlice']]
        df = df.setindex(recordID)
        
        for i in df.index:
            command = ('UPDATE {0} SET Month = {1}, TimeSlice = {2} ' + 
                      'WHERE {3} = {4}').format(table,df.loc[i,'Month'],df.loc[i,'TimeSlice'],recordID = i)
            connection.execute(command)
            
        


def imputeTimeSliceSample(dataframe):
        a = dataframe
        a.loc[:,'imputed'] = False
        initial = a.loc[:,'TimeSlice']
        
        if a.shape[0] < 2:
            return a
            
        if (a.Issue.notnull() & a.Volume.notnull() & a.TimeSlice.notnull()).sum() >= 3:
            imputable = (a.Issue.notnull() & a.Volume.notnull() & a.TimeSlice.isnull()).sum()
            if imputable > 0:
                result = sm.ols(formula="TimeSlice ~ Volume + Issue", data=a).fit()
                target = 'TimeSlice'
        elif (a.Issue.notnull() & a.Month.notnull()).sum() >= 2:
            imputable = (a.Issue.notnull() & a.Month.isnull()).sum()
            if imputable > 0:
                result = sm.ols(formula="Month ~ Issue", data=a).fit()
                target = 'Month'
        elif (a.Volume.notnull() & a.TimeSlice.notnull()).sum() >= 2:
            imputable = (a.Volume.notnull() & a.TimeSlice.isnull()).sum()
            if imputable > 0:
                result = sm.ols(formula="TimeSlice ~ Volume", data=a).fit()
                target = 'TimeSlice'
        else:
            return a
        
        if result.rsquared > confidenceThreshold:
            p = a.apply(pred,axis =1,**{'params':result.params,'target':target})
            a.loc[:,target] = p
            print(a)
            if target == 'Month':
                a.loc[:,'TimeSlice'] = a.apply(lambda row: 12*(row.loc['Year'] - 1990) + row.loc['Month'])
            
            a.loc[:,'imputed'] = a.TimeSlice != initial
        
        return imputed
            

In [110]:
os.getcwd()

'/home/matt/Git/PGIP'

In [111]:
c = alchemy.create_engine('sqlite3:////database/main.db')

NameError: name 'alchemy' is not defined